In [ ]:
from astropy.io import fits, ascii
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline
from astropy.coordinates import SkyCoord
from astropy import units as u


In [ ]:
fileL = glob.glob('all_phot/*.fits')

In [ ]:
filt, cluster, racen, deccen = [], [], [], []
for oneFile in fileL:
    HDUList = fits.open(oneFile)
    head = HDUList[1].header
    filt.append(head['FILTER'])
    cluster.append(head['OBJECT'])
    racen.append(head['RABASE'])
    deccen.append(head['DECBASE'])
t = Table()
t['File'] = fileL
t['Source'] = cluster
t['Filter'] = filt
t['RA cen'] = racen
t['DEC cen'] = deccen
t.write('ukirt_images_summary.csv')
t

In [ ]:
fL = []
fL.append('all_phot/w20161025_01764_sf_st_cat.fits')
fL.append('all_phot/w20161025_01801_sf_st_cat.fits')
fL.append('all_phot/w20161025_01838_sf_st_cat.fits')
cols = ['J','H','K']
totPhot = Table()

Doesn't work yet because I need to cross-match catalogs

In [ ]:

# for oneCol,oneFile in zip(cols,fL):
#     HDUList = fits.open(oneFile)
#     dat = Table(HDUList[1].data)
#     totPhot[oneCol] = dat['Aper_flux_3']
#     totPhot[oneCol+'err'] = dat['Aper_flux_3_err']

Mike Irwin suggested 1arcsec radius aperture photometry - aper_flux_3 for absolute flux calibration

In [ ]:
#oneFile = 'all_phot/w20161025_01838_sf_st_cat.fits'
oneFile = 'all_phot/w20161026_02024_sf_st_cat.fits'

HDUList = fits.open(oneFile)
head0 = HDUList[0].header
head1 = HDUList[1].header
dat1 = Table(HDUList[1].data)
#head1 = fits.getheader(oneFile)

In [ ]:
catalog = SkyCoord(ra=dat1['RA'] * u.radian,dec=dat1['DEC'] * u.radian)

In [ ]:
# A G2 V star
#target = SkyCoord(ra=114.593583333 * u.degree,dec=21.6154080556 * u.degree)

In [ ]:
#target = SkyCoord(ra=114.571095833 * u.degree,dec=21.6060658333 * u.degree)

In [ ]:
target = SkyCoord(ra=114.511483333 * u.degree,dec=21.5158444444 * u.degree)

In [ ]:
target.ra.hms, target.dec.dms

In [ ]:
idx, d2d, d3d = target.match_to_catalog_sky(catalog)  

In [ ]:
d2d.to(u.arcsecond)

In [ ]:
fl = dat1[idx]['Aper_flux_3']

In [ ]:
extCorr = -1. * head1['EXTINCT'] * ((head1['AMSTART']+head1['AMEND'])/2.-1.)
texpCorr = 2.5 * np.log10(head1['EXP_TIME'])
m = -2.5 * np.log10(fl) + head['MAGZPT'] + extCorr + texpCorr - head1['APCOR3']
m

In [ ]:
## Double check the coordinates

In [ ]:
dat1[idx]['RA'] * 180./np.pi, dat1[idx]['DEC'] * 180./np.pi

In [ ]:
fl, extCorr, texpCorr, head1['APCOR3']

In [ ]:
2.5 * np.log10(2. * 94e3/0.6e3)

In [ ]:
head['MAGZPT']

In [ ]:
def get_mag(fl):
    extCorr = -1. * head1['EXTINCT'] * ((head1['AMSTART']+head1['AMEND'])/2.-1.)
    texpCorr = 2.5 * np.log10(head1['EXP_TIME'])
    m = -2.5 * np.log10(fl) + head['MAGZPT'] + extCorr + texpCorr - head1['APCOR3']
    return m

In [ ]:
gDat = ascii.read('g_starsNGC_2420_copy.csv')
kMags = []
for oneRow in gDat:
    target = SkyCoord(ra=oneRow['ra'] * u.degree,dec=oneRow['dec'] * u.degree)
    idx, d2d, d3d = target.match_to_catalog_sky(catalog)
    fl = dat1[idx]['Aper_flux_3']
    kMag = get_mag(fl)
    kMags.append(kMag)
gDat['K Mag'] = kMags
gDat

Average G2 V K magnitude

In [ ]:
g2Pt = gDat['SpType'] == 'G2 V'
np.mean(gDat[g2Pt]['K Mag'])

In [ ]:
len(catalog)

In [ ]:
head1